<a href="https://colab.research.google.com/github/a1ire3a/Covid19-Datasets-Classification/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import timeit
import cv2
import glob
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import tensorflow as tf
from keras import Input
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, AveragePooling2D
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***Data Preprocessing***

In [ ]:
train_image = []
train_label = []
test_image = []
test_label = []

train_image_cnn = []
test_image_cnn = []

train_image_gcn = []
test_image_gcn = []

for filename in glob.glob('/content/drive/MyDrive/Colab Notebooks/Covid19-dataset/train/Covid/*'):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    res = cv2.resize(gray, (300, 300))
    train_image_cnn.append(res)
    arr = np.array(res)
    flat_arr = arr.ravel()
    train_image.append(flat_arr)
    train_label.append(0)

    res_gcn = cv2.resize(img,(300,300))
    train_image_gcn.append(res_gcn)
    
for filename in glob.glob('/content/drive/MyDrive/Colab Notebooks/Covid19-dataset/train/Normal/*'):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    res = cv2.resize(gray, (300, 300))
    train_image_cnn.append(res)
    arr = np.array(res)
    flat_arr = arr.ravel()
    train_image.append(flat_arr)
    train_label.append(1)

    res_gcn = cv2.resize(img,(300,300))
    train_image_gcn.append(res_gcn)
    
for filename in glob.glob('/content/drive/MyDrive/Colab Notebooks/Covid19-dataset/train/Viral Pneumonia/*'):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    res = cv2.resize(gray, (300, 300))
    train_image_cnn.append(res)
    arr = np.array(res)
    flat_arr = arr.ravel()
    train_image.append(flat_arr)
    train_label.append(2)

    res_gcn = cv2.resize(img,(300,300))
    train_image_gcn.append(res_gcn)
    
for filename in glob.glob('/content/drive/MyDrive/Colab Notebooks/Covid19-dataset/test/Covid/*'):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    res = cv2.resize(gray, (300, 300))
    test_image_cnn.append(res)
    arr = np.array(res)
    flat_arr = arr.ravel()
    test_image.append(flat_arr)
    test_label.append(0)

    res_gcn = cv2.resize(img,(300,300))
    test_image_gcn.append(res_gcn)
    
for filename in glob.glob('/content/drive/MyDrive/Colab Notebooks/Covid19-dataset/test/Normal/*'):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    res = cv2.resize(gray, (300, 300))
    test_image_cnn.append(res)
    arr = np.array(res)
    flat_arr = arr.ravel()
    test_image.append(flat_arr)
    test_label.append(1)

    res_gcn = cv2.resize(img,(300,300))
    test_image_gcn.append(res_gcn)
    
for filename in glob.glob('/content/drive/MyDrive/Colab Notebooks/Covid19-dataset/test/Viral Pneumonia/*'):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    res = cv2.resize(gray, (300, 300))
    test_image_cnn.append(res)
    arr = np.array(res)
    flat_arr = arr.ravel()
    test_image.append(flat_arr)
    test_label.append(2)

    res_gcn = cv2.resize(img,(300,300))
    test_image_gcn.append(res_gcn)
    
train_image = np.asarray(train_image)
train_label = np.asarray(train_label)
test_image = np.asarray(test_image)
test_label = np.asarray(test_label)

Normalize data for CNN model

In [ ]:
train_image_cnn = np.asarray(train_image_cnn)
test_image_cnn = np.asarray(test_image_cnn)
train_image_cnn = train_image_cnn/255.0
test_image_cnn = test_image_cnn/255.0

Normalize data for SVM and logistic regression

In [ ]:
train_image_std = StandardScaler().fit_transform(train_image)
test_image_std = StandardScaler().fit_transform(test_image)

SVM model initialization

In [ ]:
clf_svm = SVC(kernel='poly')

SVM train and test

In [ ]:
start = timeit.default_timer()
clf_svm.fit(train_image_std, train_label)
pred_label_svm = clf_svm.predict(test_image_std)
acc_svm = accuracy_score(test_label, pred_label_svm)
stop = timeit.default_timer()

print('Time: ', stop - start) 
print('SVM Accuracy: ', acc_svm)

Time:  6.905712615000084
SVM Accuracy:  0.8787878787878788


Logistic regression initialization

In [ ]:
clf_LR = LogisticRegression(max_iter=300)

Logistic regression train and test

In [ ]:
start = timeit.default_timer()
clf_LR.fit(train_image_std, train_label)
pred_label_LR = clf_LR.predict(test_image_std)
acc_LR = accuracy_score(test_label, pred_label_LR)
stop = timeit.default_timer()

print('Time: ', stop - start) 
print('Logistic Regresssion Accuracy: ', acc_LR)

Time:  24.81545259199993
Logistic Regresssion Accuracy:  0.8939393939393939


# ***PCA***

In [ ]:
pca = PCA(n_components=2)
train_image_std_pca = pca.fit(train_image_std).transform(train_image_std)
test_image_std_pca = pca.fit(test_image_std).transform(test_image_std)

# ***LDA***

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=2)
train_image_std_lda = lda.fit(train_image_std, train_label).transform(train_image_std)
test_image_std_lda = lda.fit(test_image_std, test_label).transform(test_image_std)

SVM with pca

In [ ]:
start = timeit.default_timer()
clf_svm.fit(train_image_std_pca, train_label)
pred_label_svm = clf_svm.predict(test_image_std_pca)
acc_svm = accuracy_score(test_label, pred_label_svm)
stop = timeit.default_timer()

print('Time: ', stop - start) 
print('SVM Accuracy with pca: ', acc_svm)

Time:  0.008689893998962361
SVM Accuracy with pca:  0.7424242424242424


Logistic regression with pca

In [ ]:
start = timeit.default_timer()
clf_LR.fit(train_image_std_pca, train_label)
pred_label_LR = clf_LR.predict(test_image_std_pca)
acc_LR = accuracy_score(test_label, pred_label_LR)
stop = timeit.default_timer()

print('Time: ', stop - start) 
print('Logistic Regresssion Accuracy pca: ', acc_LR)

Time:  0.03679871200074558
Logistic Regresssion Accuracy pca:  0.7727272727272727


SVM with LDA

In [ ]:
start = timeit.default_timer()
clf_svm.fit(train_image_std_lda, train_label)
pred_label_svm = clf_svm.predict(test_image_std_lda)
acc_svm = accuracy_score(test_label, pred_label_svm)
stop = timeit.default_timer()

print('Time: ', stop - start) 
print('SVM Accuracy with lda: ', acc_svm)

Time:  0.002268855000011172
SVM Accuracy with lda:  0.9696969696969697


Logistic regression with LDA

In [ ]:
start = timeit.default_timer()
clf_LR.fit(train_image_std_lda, train_label)
pred_label_LR = clf_LR.predict(test_image_std_lda)
acc_LR = accuracy_score(test_label, pred_label_LR)
stop = timeit.default_timer()

print('Time: ', stop - start) 
print('Logistic Regresssion Accuracy lda: ', acc_LR)

Time:  0.023233625001012115
Logistic Regresssion Accuracy lda:  0.9848484848484849


# ***GCN***

In [ ]:
import numpy
import scipy

def global_contrast_normalization(filename, s, lmda, epsilon):
    X = numpy.array(filename)

    # replacement for the loop
    X_average = numpy.mean(X)
    
    X = X - X_average

    
    contrast = numpy.sqrt(lmda + numpy.mean(X**2))

    X = s * X / max(contrast, epsilon)

    # scipy can handle it
    return X

In [ ]:
gcn_train = np.asarray([global_contrast_normalization(x,1, 0.1, 0.000000001) for x in train_image_cnn])
gcn_test = np.asarray([global_contrast_normalization(x,1, 0.1, 0.000000001) for x in test_image_cnn])

In [ ]:
train_flat = []
test_flat = []
for X in gcn_train:
    arr = np.array(X)
    flat_arr = arr.ravel()
    train_flat.append(flat_arr)

for X in gcn_test:
    arr = np.array(X)
    flat_arr = arr.ravel()
    test_flat.append(flat_arr)

train_flat = np.asarray(train_flat)
test_flat = np.asarray(test_flat)

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=2)
train_image_gcn_lda = lda.fit(train_flat, train_label).transform(train_flat)
test_image_gcn_lda = lda.fit(test_flat, test_label).transform(test_flat)

Logistic regression with gcn normalization and lda dimensionality reduction

In [ ]:
start = timeit.default_timer()
clf_LR.fit(train_image_gcn_lda, train_label)
pred_label_LR = clf_LR.predict(test_image_gcn_lda)
acc_LR = accuracy_score(test_label, pred_label_LR)
stop = timeit.default_timer()

print('Time: ', stop - start) 
print('Logistic Regresssion Accuracy lda and gcn: ', acc_LR)

Time:  0.01825196000027063
Logistic Regresssion Accuracy lda and gcn:  0.9242424242424242


SVM with gcn normalization and lda dimensionality reduction

In [ ]:
start = timeit.default_timer()
clf_svm.fit(train_image_gcn_lda, train_label)
pred_label_svm = clf_svm.predict(test_image_gcn_lda)
acc_svm = accuracy_score(test_label, pred_label_svm)
stop = timeit.default_timer()

print('Time: ', stop - start) 
print('SVM Accuracy with lda and gcn: ', acc_svm)

Time:  0.00243041699997093
SVM Accuracy with lda and gcn:  0.9545454545454546


# ***CNN***

In [ ]:
train_label_r = train_label.reshape(251,1)
test_label_r = test_label.reshape(66,1)

In [ ]:
model = Sequential()
model.add(Conv2D(149, (3, 3), activation='relu', input_shape=(300, 300, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(294, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(299, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(528, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(896, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(343, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_79"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_275 (Conv2D)         (None, 298, 298, 149)     1490      
                                                                 
 max_pooling2d_221 (MaxPooli  (None, 149, 149, 149)    0         
 ng2D)                                                           
                                                                 
 conv2d_276 (Conv2D)         (None, 147, 147, 294)     394548    
                                                                 
 max_pooling2d_222 (MaxPooli  (None, 73, 73, 294)      0         
 ng2D)                                                           
                                                                 
 conv2d_277 (Conv2D)         (None, 71, 71, 299)       791453    
                                                                 
 max_pooling2d_223 (MaxPooli  (None, 35, 35, 299)    

In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True), metrics=['accuracy'])

rep = model.fit(gcn_train, train_label_r, epochs=10, validation_data=(gcn_test, test_label_r))

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


8/8 [==============================] - 14s 2s/step - loss: 1.2945 - accuracy: 0.4661 - val_loss: 0.9667 - val_accuracy: 0.6970
Epoch 2/10
8/8 [==============================] - 12s 2s/step - loss: 0.6394 - accuracy: 0.7410 - val_loss: 0.4596 - val_accuracy: 0.8182
Epoch 3/10
8/8 [==============================] - 12s 2s/step - loss: 0.3690 - accuracy: 0.8446 - val_loss: 0.5514 - val_accuracy: 0.7121
Epoch 4/10
8/8 [==============================] - 13s 2s/step - loss: 0.2403 - accuracy: 0.9124 - val_loss: 0.4384 - val_accuracy: 0.9091
Epoch 5/10
8/8 [==============================] - 13s 2s/step - loss: 0.2085 - accuracy: 0.9203 - val_loss: 0.4043 - val_accuracy: 0.9091
Epoch 6/10
8/8 [==============================] - 13s 2s/step - loss: 0.1660 - accuracy: 0.9323 - val_loss: 0.4293 - val_accuracy: 0.8636
Epoch 7/10
8/8 [==============================] - 13s 2s/step - loss: 0.1300 - accuracy: 0.9522 - val_loss: 0.5149 - val_accuracy: 0.9242
Epoch 8/10
8/8 [=============================

In [ ]:
test_loss, test_acc = model.evaluate(gcn_test, test_label_r)
print(test_acc)

3/3 [==============================] - 1s 236ms/step - loss: 0.3978 - accuracy: 0.9091
0.9090909361839294
